In [1]:
#-------------------------------------------------------Data preparing and PCA-------------------------------------------------------#
import numpy as np
import time
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score
from sklearn.preprocessing import StandardScaler
from keras.datasets import cifar10

# Load CIFAR-10 into our programm
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# Class labels in CIFAR-10 dataset to make the project more general
labels = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

# Select the two desired classes
class_1, class_2 = 3, 5
mask_train = (y_train.flatten() == class_1) | (y_train.flatten() == class_2)
mask_test = (y_test.flatten() == class_1) | (y_test.flatten() == class_2)

x_train, y_train = x_train[mask_train], y_train[mask_train]
x_test, y_test = x_test[mask_test], y_test[mask_test]

# Relabel classes to 0 and 1 
y_train = (y_train.flatten() == class_2).astype(int)
y_test = (y_test.flatten() == class_2).astype(int)


# Reshaping our data to fit sckit learn SVM requirments
x_train_flat = x_train.reshape(x_train.shape[0], -1)
x_test_flat = x_test.reshape(x_test.shape[0], -1)

# Standardize our data 
scaler = StandardScaler()
x_train_flat = scaler.fit_transform(x_train_flat)
x_test_flat = scaler.transform(x_test_flat)


# Apply PCA
n_components = 0.9 
pca = PCA(n_components=n_components)
x_train_pca = pca.fit_transform(x_train_flat)
x_test_pca = pca.transform(x_test_flat)




In [ ]:
#-------------------------------------------------------Linear SVM-------------------------------------------------------#

C = [0.1, 1]

for i in C:
    start_time = time.perf_counter()    
    svm_model = SVC(kernel='linear', C=i)
    svm_model.fit(x_train_pca, y_train)
    y_pred_train = svm_model.predict(x_train_pca)  
    y_pred = svm_model.predict(x_test_pca)
    end_time = time.perf_counter()
    execution_time = end_time - start_time

    print(f"\nSVM Classification Report with C: {i}:")
    print(f"Training accuracy: {accuracy_score(y_train, y_pred_train)*100}%")
    print(f"Test accuracy: {accuracy_score(y_test, y_pred)*100}%")
    print(f"Execution time: {execution_time:.5f} seconds")



In [ ]:
#-------------------------------------------------------RBF SVM-------------------------------------------------------#

C = [0.00001, 0.0001, 0.001, 0.01, 0.1]
GAMMA = [0.00001, 0.0001, 0.001, 0.01, 0.1]


for i in C:
    for j in GAMMA:
        start_time = time.perf_counter()  
        svm_model = SVC(kernel='rbf', gamma=j, C=i)
        svm_model.fit(x_train_pca, y_train)
        y_train_pred = svm_model.predict(x_train_pca)
        train_accuracy = accuracy_score(y_train, y_train_pred)
        y_pred = svm_model.predict(x_test_pca)
        test_accuracy = accuracy_score(y_test, y_pred)
        end_time = time.perf_counter()
        execution_time = end_time - start_time
        
        print(f"\nSVM Classification Report with C: {i}, gamma: {j}:")
        print(f"Training Accuracy: {train_accuracy * 100:.2f}%")
        print(f"Testing Accuracy: {test_accuracy * 100:.2f}%")
        print(f"Execution time: {execution_time:.5f} seconds")




In [ ]:
#-------------------------------------------------------Polynomial SVM-------------------------------------------------------#

C = [0.001, 0.01, 0.1, 1]

G = [0.001, 0.01, 0.1, 1, 10]
DEGREE = [2, 3, 4]  # Add degrees for the polynomial kernel

for i in C:
    for j in G:
        for d in DEGREE:
            start_time = time.perf_counter()  
            svm_model = SVC(kernel='poly', gamma=j, C=i, degree=d)
            svm_model.fit(x_train_pca, y_train)
            
            y_train_pred = svm_model.predict(x_train_pca)
            train_accuracy = accuracy_score(y_train, y_train_pred)
            y_pred = svm_model.predict(x_test_pca)
            test_accuracy = accuracy_score(y_test, y_pred)            
            end_time = time.perf_counter()
            execution_time = end_time - start_time
            
            print(f"\nSVM Classification Report with C: {i}, gamma: {j}, degree: {d}:")
            print(f"Training accuracy: {train_accuracy * 100:.2f}%")
            print(f"Testing accuracy: {test_accuracy * 100:.2f}%")
            print(f"Execution time: {execution_time:.5f} seconds")



In [ ]:
#-------------------------------------------------------MLP with Hinge Loss-------------------------------------------------------#
from keras.models import Sequential
from keras.layers import Dense, Input, LeakyReLU
from keras.optimizers import Adam
from keras.losses import hinge
from keras.callbacks import LearningRateScheduler
import numpy as np



model = Sequential([
    Input(shape=(x_train_pca.shape[1],)),  # Input layer with shape eequal to x_train_pca array
    Dense(256),                            # Hidden layer with 128 neurons
    LeakyReLU(alpha=0.01),                 # Leaky ReLU activation with a negative slope of 0.01
    Dense(1, activation='sigmoid')          # Output layer with linear activation for hinge loss
])

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.0001), loss=hinge, metrics=['accuracy'])

# Learning rate scheduler function
def lr_schedule(epoch):
    if epoch <= 8:
        return 0.0001
    else:
        return 0.0001 * 0.75

# Learning rate scheduler callback
lr_scheduler = LearningRateScheduler(lr_schedule)

# Train our model
start_time = time.time()
history = model.fit(x_train_pca, y_train, epochs=25 ,batch_size=32, validation_data=(x_test_pca, y_test), callbacks=[lr_scheduler])
end_time = time.time()

print(f"Training time: {end_time - start_time:.2f} seconds")

# Print our models test accuracy
_, accuracy = model.evaluate(x_test_pca, y_test, verbose=0)
print(f"Model Accuracy: {accuracy*100:.2f}%")
